In [1]:
from utils import *
from pathlib import Path
from typing import NamedTuple, Tuple
import pandas as pd
import boto3
from rekognition_objects import RekognitionFace

## Setup

In [2]:
val_df = get_valdf()
images = get_images()

In [3]:
class FaceAnalysis(NamedTuple):
    path: str
    age: Tuple[int, int]
    gender: str

In [4]:
rekognition_client = boto3.client('rekognition')

## Function for analyzing images

In [5]:
def aws_analyze_image(path: Path) -> FaceAnalysis:
    short_path = '/'.join(str(path).split('/')[-2:])
    try:
        with open(path, 'rb') as image_data:
            image = {'Bytes': image_data.read()}
            response = rekognition_client.detect_faces(Image = image, Attributes = ['ALL'])
            faces = [RekognitionFace(face) for face in response['FaceDetails']]
            face = faces[0]
            return FaceAnalysis(path=short_path, age=face.age_range, gender=face.gender)
    except:
        return FaceAnalysis(path=short_path, age=None, gender=None)

## Building the dataframe

In [6]:
paths, ages, genders = [], [], []

In [7]:
for idx, im in enumerate(images):
    if idx % 200 == 0:
        print(idx)
    face = aws_analyze_image(im)
    paths.append(face.path)
    ages.append(face.age)
    genders.append(face.gender)

0
200
400
600
800
1000
1200
1400
1600
1800
2000
2200
2400
2600
2800
3000
3200
3400
3600
3800
4000
4200
4400
4600
4800
5000
5200
5400
5600
5800
6000
6200
6400
6600
6800
7000
7200
7400
7600
7800
8000
8200
8400
8600
8800
9000
9200
9400
9600
9800
10000
10200
10400
10600
10800


In [8]:
analyzed_df = pd.DataFrame({'path': paths, 'predicted_age': ages, 'predicted_gender': genders})

In [9]:
analyzed_df['predicted_age'].isna().sum()

156

In [10]:
analyzed_df.head()

,path,predicted_age,predicted_gender
0,val/9733.jpg,"(23, 37)",Male
1,val/63.jpg,"(4, 14)",Female
2,val/6400.jpg,"(25, 39)",Female
3,val/823.jpg,"(26, 40)",Male
4,val/4217.jpg,"(2, 8)",Female


In [12]:
val_df.merge(analyzed_df, left_on = 'file', right_on = 'path').to_csv('aws_val_merged.csv', index = False)